In [1]:
import os
import sys
import pickle
import numpy as np
from numpy import gradient
from math import pi
from packages import data_container
from packages.data_container import Data
from packages.helper import traj_speed, beta, min_dist, d_psi, min_sep
# For pickle to load the Data object, which is defined in packages.data_container
sys.modules['data_container'] = data_container

In [53]:
'''Import data from csv to Data object'''
header=('subj_x', 'subj_y', 'subj_z', 'subj_yaw', 'subj_pitch', 'subj_row')
info={'p_goal':[], 'subj_id':[], 'trial_id':[], 'stimuli_onset':[],  'goal_s0':[], 'goal_d0':[], 'goal_angle':[],
      'stimuli_out':[], 'w_goal': 0.4, 'ps_trial':[], 'ps_subj':[], 'time_stamp':[],
      'p_subj':[], 'v_subj':[], 'a_subj':[]}
data = Data(Hz=90, header=header, info=info)
n_subj = 1
for subj_id in range(n_subj):
    # Import output
    file_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw_Data', 'Bai_moveObst1b_approach'
                                             'Subj' + str(subj_id).zfill(2) + '.csv'))
    with open(file_path, 'r') as f:
        trials = [line.split(',') for line in f.read().split('\n')]
    head = 0
    while head + 1 < len(trials):
        for i in range(head+1, len(trials)):
            if trials[i][0] == '':
                tail = i - 1
                break
        trial_id = int(trials[head+1][1])
        time_stamp = [float(line[0]) for line in trials[head+3:tail+1]]
        s0 = float(trials[head+1][3])
        d0 = float(trials[head+1][5])
        angle = float(trials[head+1][7])
        subj = np.array([[float(val) for val in line[1:7]] for line in trials[head+3:tail+1]])
        p0 = Data.filter(subj[:,[0,2]], time_stamp, 90, 4, 0.6)
        v0 = gradient(p0, axis=0) * data.Hz
        a0 = gradient(v0, axis=0) * data.Hz
        goal = np.array([[float(line[-3]), float(line[-2])] for line in trials[head+3:tail+1]])
        if s0:
            for i in range(head+3, tail+1):
                if trials[i][-1] == 'TRUE':
                    stimuli_onset = i - head - 3
                    break
            stimuli_out = len(time_stamp) - 1
        else:
            stimuli_onset = stimuli_out = None
        data.add_traj(subj)
        ps_trial = np.mean(traj_speed(subj[-2*data.Hz:-data.Hz,[0,2]], data.Hz))
        data.add_info({'p_goal':goal, 'subj_id':subj_id, 'trial_id':trial_id,
                       'stimuli_onset':stimuli_onset, 'stimuli_out':stimuli_out, 'time_stamp':time_stamp, 
                       'ps_trial':ps_trial, 'ps_subj':0, 'goal_s0':s0, 'goal_d0':d0, 'goal_angle':angle,
                       'p_subj':p0, 'v_subj':v0, 'a_subj':a0})
        head = tail + 1
ps_subj = [0] * n_subj
for i in range(len(data.trajs)):
    if data.info['goal_s0'][i] == 0:
        subj_id = data.info['subj_id'][i]
        ps_subj[subj_id] += data.info['ps_trial'][i] / 10  # Because there were 10 freewalk trials
for i in range(len(data.trajs)):
    subj_id = data.info['subj_id'][i]
    data.info['ps_subj'][i] = ps_subj[subj_id]
outfile = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw_Data', 'Bai_movObst1b_data.pickle'))       
with open(outfile, 'wb') as f:   
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)  

In [52]:
i = 1
print(data.info['stimuli_onset'][i])
print(data.info['time_stamp'][i][297])
print(data.info['stimuli_out'][i], len(data.info['time_stamp'][i]))
print(data.info['goal_s0'][i], data.info['goal_d0'][i], data.info['goal_angle'][i])
print(data.info['ps_trial'][i], data.info['ps_subj'][i])
print(data.info['p_subj'][i])
print(data.info['v_subj'][i][-180:-90])
print(data.info['a_subj'][i])

297
3.3109257
776 777
1.4 8.0 15.0
1.3432401114908668 1.2848989666515145
[[ 4.42469469  5.3645755 ]
 [ 4.42423989  5.36484124]
 [ 4.4237709   5.36509717]
 ...
 [-3.03548671 -0.9643004 ]
 [-3.04730715 -0.97178725]
 [-3.05912224 -0.97924831]]
[[-1.13899604 -0.69257195]
 [-1.13973961 -0.69165587]
 [-1.14048155 -0.69074942]
 [-1.14122109 -0.68985323]
 [-1.14195753 -0.68896783]
 [-1.14269015 -0.68809372]
 [-1.1434183  -0.68723134]
 [-1.14414132 -0.68638107]
 [-1.14485862 -0.68554323]
 [-1.14556961 -0.68471811]
 [-1.14627374 -0.68390592]
 [-1.14697052 -0.68310684]
 [-1.14765946 -0.68232096]
 [-1.14834011 -0.68154837]
 [-1.14901206 -0.68078908]
 [-1.14967494 -0.68004306]
 [-1.1503284  -0.67931023]
 [-1.15097212 -0.67859049]
 [-1.15160582 -0.67788367]
 [-1.15222924 -0.67718961]
 [-1.15284217 -0.67650806]
 [-1.1534444  -0.6758388 ]
 [-1.15403576 -0.67518155]
 [-1.1546161  -0.67453601]
 [-1.15518529 -0.67390188]
 [-1.15574324 -0.67327883]
 [-1.15628984 -0.67266654]
 [-1.15682504 -0.67206466]
 [-